In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
import hdbscan
import seaborn as sns
sns.set_style("dark")

In [2]:
pd.set_option('max_colwidth', 1000)

# Load Data

Here, I'm reading in the imputed data set Julia put together, and pulling in some metadata from the non-imputed data.

In [3]:
data = pd.read_csv('data/scorecard_reduced_features.csv')
imputed = pd.read_csv('data/scorecard_imputed.csv')
imputed['UNITID'] = data.UNITID

Set the index to the UNITID, and inspect the dataframes:

In [4]:
data.set_index('UNITID', inplace=True)
data.head()

,INSTNM,ZIP,HCM2,CONTROL,LOCALE,CCBASIC,CCUGPROF,CCSIZSET,HBCU,PBI,...,LO_INC_RPY_3YR_RT_SUPP,MD_INC_RPY_3YR_RT_SUPP,HI_INC_RPY_3YR_RT_SUPP,NONCOM_RPY_3YR_RT_SUPP,FIRSTGEN_RPY_3YR_RT_SUPP,PCT_LIBERAL_ARTS,PCT_PROFESSIONAL,PCT_RELIGIOUS,COST,PCT_VOCATIONAL
UNITID,,,,,,,,,,,,,,,,,,,,,
100654,Alabama A & M University,35762,0.0,1.0,12.0,18.0,9.0,14.0,1.0,0.0,...,0.448163,0.446488,0.422018,0.373790,0.423581,0.5346,0.4074,0.0000,18888.0,0.0629
100663,University of Alabama at Birmingham,35294-0110,0.0,1.0,12.0,15.0,8.0,15.0,0.0,0.0,...,0.724256,0.770134,0.818059,0.678749,0.751553,0.4379,0.5805,0.0095,19990.0,0.0000
100690,Amridge University,36117-3553,0.0,2.0,12.0,21.0,6.0,6.0,0.0,0.0,...,0.597701,0.709091,NaN,0.613281,0.632653,0.6301,0.1096,0.2603,12300.0,0.0000
100706,University of Alabama in Huntsville,35899,0.0,1.0,12.0,15.0,8.0,12.0,0.0,0.0,...,0.721404,0.824742,0.851936,0.689788,0.778157,0.2764,0.7152,0.0082,20306.0,0.0000
100724,Alabama State University,36104-0271,0.0,1.0,12.0,18.0,9.0,13.0,1.0,0.0,...,0.310875,0.376106,0.333333,0.315997,0.315737,0.5449,0.5733,0.0000,17400.0,0.0000


In [5]:
imputed.set_index('UNITID', inplace=True)
imputed.head()

,HCM2,CONTROL,LOCALE,CCBASIC,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,...,LO_INC_RPY_3YR_RT_SUPP,MD_INC_RPY_3YR_RT_SUPP,HI_INC_RPY_3YR_RT_SUPP,NONCOM_RPY_3YR_RT_SUPP,FIRSTGEN_RPY_3YR_RT_SUPP,PCT_LIBERAL_ARTS,PCT_VOCATIONAL,PCT_RELIGIOUS,COST,PCT_PROFESSIONAL
UNITID,,,,,,,,,,,,,,,,,,,,,
100654,0,1,12,18,1,0,0,0,0,0,...,0.448163,0.446488,0.422018,0.373790,0.423581,0.5346,0.0629,0.0000,18888.0,0.4074
100663,0,1,12,15,0,0,0,0,0,0,...,0.724256,0.770134,0.818059,0.678749,0.751553,0.4379,0.0000,0.0095,19990.0,0.5805
100690,0,2,12,21,0,0,0,0,0,0,...,0.597701,0.709091,0.788065,0.613281,0.632653,0.6301,0.0000,0.2603,12300.0,0.1096
100706,0,1,12,15,0,0,0,0,0,0,...,0.721404,0.824742,0.851936,0.689788,0.778157,0.2764,0.0000,0.0082,20306.0,0.7152
100724,0,1,12,18,1,0,0,0,0,0,...,0.310875,0.376106,0.333333,0.315997,0.315737,0.5449,0.0000,0.0000,17400.0,0.5733


Everything matches up. Good!


We can use `pandas.get_dummies()` to break out the categorical columns and improve the performace of our cosine similarity metric.

In [6]:
imputed = pd.get_dummies(imputed, columns=['CONTROL', 'LOCALE', 'CCBASIC'])#, 'CCUGPROF', 'CCSIZSET'])

Then, to make sure all features are treated equally, normalize them to the range 0-1.

In [7]:
rescaled = MinMaxScaler().fit_transform(imputed)
imputed[imputed.columns] = rescaled

In [8]:
rescaled.shape

(2473, 93)

# Compute the cosine similarity:

Import from scikit-learn and let it do all the work :-)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

c = cosine_similarity(rescaled, rescaled)

That returns a square numpy array with the similarity of each school to every other.  Now we need to pipe it back into a dataframe so we can keep everything indexed:

In [10]:
cosim = pd.DataFrame(c, index=data.index, columns=data.index)
cosim['INSTNM'] = data.INSTNM
cosim['ZIP'] = data.ZIP

Rearrange the columns so INSTNM and ZIP are at the front:

In [11]:
cols = cosim.columns.tolist()
cosim = cosim[cols[-2:]+cols[:-2]]

cosim.head()

UNITID,INSTNM,ZIP,100654,100663,100690,100706,100724,100751,100812,100830,...,45891904,45891905,45891906,45891907,45896401,45896402,45897301,45897302,45897303,45897304
UNITID,,,,,,,,,,,,,,,,,,,,,
100654,Alabama A & M University,35762,1.000000,0.751817,0.649907,0.721509,0.928033,0.622077,0.618940,0.849199,...,0.481003,0.481003,0.481003,0.481003,0.481003,0.481003,0.481003,0.481003,0.481003,0.481003
100663,University of Alabama at Birmingham,35294-0110,0.751817,1.000000,0.790216,0.991159,0.663715,0.842650,0.811736,0.878556,...,0.586366,0.586366,0.586366,0.586366,0.586366,0.586366,0.586366,0.586366,0.586366,0.586366
100690,Amridge University,36117-3553,0.649907,0.790216,1.000000,0.764079,0.589172,0.693059,0.706366,0.769680,...,0.605232,0.605232,0.605232,0.605232,0.605232,0.605232,0.605232,0.605232,0.605232,0.605232
100706,University of Alabama in Huntsville,35899,0.721509,0.991159,0.764079,1.000000,0.629289,0.837913,0.815738,0.866514,...,0.588891,0.588891,0.588891,0.588891,0.588891,0.588891,0.588891,0.588891,0.588891,0.588891
100724,Alabama State University,36104-0271,0.928033,0.663715,0.589172,0.629289,1.000000,0.522988,0.539998,0.785523,...,0.449587,0.449587,0.449587,0.449587,0.449587,0.449587,0.449587,0.449587,0.449587,0.449587


# Look at a few examples:

In [12]:
cosim[cosim.INSTNM.str.contains('Harvard')]['INSTNM']

UNITID
166027    Harvard University
Name: INSTNM, dtype: object

In [13]:
cosim[['INSTNM', 166027]].sort_values(166027, ascending=False).head(15)

UNITID,INSTNM,166027
UNITID,,
166027,Harvard University,1.000000
130794,Yale University,0.991084
198419,Duke University,0.983294
217156,Brown University,0.980329
166683,Massachusetts Institute of Technology,0.979921
195030,University of Rochester,0.955522
110404,California Institute of Technology,0.949982
243744,Stanford University,0.947793
182670,Dartmouth College,0.944073


In [14]:
cosim[cosim.INSTNM.str.contains('Truman')]['INSTNM']

UNITID
178615    Truman State University
Name: INSTNM, dtype: object

In [15]:
cosim[['INSTNM', 178615]].sort_values(178615, ascending=False).head(15)

UNITID,INSTNM,178615
UNITID,,
178615,Truman State University,1.000000
232566,Longwood University,0.986828
171456,Northern Michigan University,0.971974
221768,The University of Tennessee-Martin,0.964441
175078,Southwest Minnesota State University,0.959259
115755,Humboldt State University,0.954898
200253,Minot State University,0.936694
107983,Southern Arkansas University Main Campus,0.935894
207865,Southwestern Oklahoma State University,0.927232


In [16]:
cosim[cosim.INSTNM.str.contains('University of Utah')]['INSTNM']

UNITID
230764    University of Utah
Name: INSTNM, dtype: object

In [17]:
cosim[['INSTNM', 230764]].sort_values(230764, ascending=False).head(15)

UNITID,INSTNM,230764
UNITID,,
230764,University of Utah,1.000000
126818,Colorado State University-Fort Collins,0.991681
221759,The University of Tennessee-Knoxville,0.990772
104151,Arizona State University-Tempe,0.990071
209551,University of Oregon,0.986680
178396,University of Missouri-Columbia,0.985976
200332,North Dakota State University-Main Campus,0.985357
234030,Virginia Commonwealth University,0.984573
218663,University of South Carolina-Columbia,0.983945


In [18]:
cosim[cosim.INSTNM.str.contains('University of Texas at Austin')]['INSTNM']

UNITID
228778    The University of Texas at Austin
Name: INSTNM, dtype: object

In [19]:
cosim[['INSTNM', 228778]].sort_values(228778, ascending=False).head(15)

UNITID,INSTNM,228778
UNITID,,
228778,The University of Texas at Austin,1.000000
236948,University of Washington-Seattle Campus,0.993856
110680,University of California-San Diego,0.992566
174066,University of Minnesota-Twin Cities,0.991074
139755,Georgia Institute of Technology-Main Campus,0.990308
110662,University of California-Los Angeles,0.990089
204796,Ohio State University-Main Campus,0.989203
199193,North Carolina State University at Raleigh,0.986759
137351,University of South Florida-Main Campus,0.986201


In [20]:
cosim[cosim.INSTNM.str.contains('Missouri University of Science and Technology')]['INSTNM']

UNITID
178411    Missouri University of Science and Technology
Name: INSTNM, dtype: object

In [21]:
cosim[['INSTNM', 178411]].sort_values(178411, ascending=False).head(15)

UNITID,INSTNM,178411
UNITID,,
178411,Missouri University of Science and Technology,1.000000
171128,Michigan Technological University,0.976027
219356,South Dakota State University,0.958949
176017,University of Mississippi,0.958815
240727,University of Wyoming,0.957252
100858,Auburn University,0.918862
151111,Indiana University-Purdue University-Indianapolis,0.898489
180461,Montana State University,0.897983
155399,Kansas State University,0.895094


In [22]:
cosim[cosim.INSTNM.str.contains('California Institute of Technology')]['INSTNM']

UNITID
110404    California Institute of Technology
Name: INSTNM, dtype: object

In [23]:
cosim[['INSTNM', 110404]].sort_values(110404, ascending=False).head(15)

UNITID,INSTNM,110404
UNITID,,
110404,California Institute of Technology,1.000000
166683,Massachusetts Institute of Technology,0.975299
195030,University of Rochester,0.963670
217156,Brown University,0.962175
198419,Duke University,0.961115
130794,Yale University,0.959402
166027,Harvard University,0.949982
211440,Carnegie Mellon University,0.925095
227757,Rice University,0.924326


In [24]:
cosim[cosim.INSTNM.str.contains('Stanford')]['INSTNM']

UNITID
243744    Stanford University
Name: INSTNM, dtype: object

In [25]:
cosim[['INSTNM', 243744]].sort_values(243744, ascending=False).head(15)

UNITID,INSTNM,243744
UNITID,,
243744,Stanford University,1.000000
186131,Princeton University,0.985658
179867,Washington University in St Louis,0.984672
139658,Emory University,0.983616
168148,Tufts University,0.982359
152080,University of Notre Dame,0.981309
135726,University of Miami,0.978513
130794,Yale University,0.952529
215062,University of Pennsylvania,0.950277


In [26]:
cosim[cosim.INSTNM.str.contains('Brigham Young')]['INSTNM']

UNITID
142522     Brigham Young University-Idaho
230038     Brigham Young University-Provo
230047    Brigham Young University-Hawaii
Name: INSTNM, dtype: object

In [27]:
cosim[['INSTNM', 230038]].sort_values(230038, ascending=False).head(15)

UNITID,INSTNM,230038
UNITID,,
230038,Brigham Young University-Provo,1.000000
223232,Baylor University,0.971371
199847,Wake Forest University,0.970221
165334,Clark University,0.970007
202480,University of Dayton,0.963099
196413,Syracuse University,0.960718
21997601,Spark,0.913005
229115,Texas Tech University,0.911639
143358,Bradley University,0.910920


In [28]:
cosim[cosim.INSTNM.str.contains('Calvin College')]

UNITID,INSTNM,ZIP,100654,100663,100690,100706,100724,100751,100812,100830,...,45891904,45891905,45891906,45891907,45896401,45896402,45897301,45897302,45897303,45897304
UNITID,,,,,,,,,,,,,,,,,,,,,
169080,Calvin College,49546,0.635261,0.827865,0.893868,0.821693,0.535021,0.773005,0.736335,0.793831,...,0.544573,0.544573,0.544573,0.544573,0.544573,0.544573,0.544573,0.544573,0.544573,0.544573


In [29]:
cosim[['INSTNM', 169080]].sort_values(169080, ascending=False).head(15)

UNITID,INSTNM,169080
UNITID,,
169080,Calvin College,1.000000
175980,Millsaps College,0.984742
153144,Coe College,0.980639
236328,University of Puget Sound,0.978813
210401,Willamette University,0.976808
158477,Centenary College of Louisiana,0.975647
130590,Trinity College,0.965384
233374,University of Richmond,0.963268
100937,Birmingham Southern College,0.960006


In [30]:
cosim[cosim.INSTNM.str.contains('Spark')]

UNITID,INSTNM,ZIP,100654,100663,100690,100706,100724,100751,100812,100830,...,45891904,45891905,45891906,45891907,45896401,45896402,45897301,45897302,45897303,45897304
UNITID,,,,,,,,,,,,,,,,,,,,,
21997601,Spark,370674896,0.617134,0.830971,0.843138,0.830644,0.520759,0.769498,0.731664,0.775266,...,0.555322,0.555322,0.555322,0.555322,0.555322,0.555322,0.555322,0.555322,0.555322,0.555322


In [31]:
cosim[['INSTNM', 21997601]].sort_values(21997601, ascending=False).head(15)

UNITID,INSTNM,21997601
UNITID,,
21997601,Spark,1.000000
16822703,Wentworth Institute of Technology,0.996240
16822702,Wentworth Institute of Technology,0.996240
16822701,Wentworth Institute of Technology,0.996240
233842,Union Presbyterian Seminary,0.995181
459736,Touro University California,0.994521
120795,Pacific School of Religion,0.994360
165705,Episcopal Divinity School,0.994082
169099,Calvin Theological Seminary,0.994061


In [32]:
cosim[cosim.INSTNM.str.contains('Lipscomb')]

UNITID,INSTNM,ZIP,100654,100663,100690,100706,100724,100751,100812,100830,...,45891904,45891905,45891906,45891907,45896401,45896402,45897301,45897302,45897303,45897304
UNITID,,,,,,,,,,,,,,,,,,,,,
219976,Lipscomb University,37204-3951,0.634802,0.755495,0.756662,0.7517,0.541427,0.76086,0.730206,0.790897,...,0.55573,0.55573,0.55573,0.55573,0.55573,0.55573,0.55573,0.55573,0.55573,0.55573


In [33]:
cosim[['INSTNM', 219976]].sort_values(219976, ascending=False).head(15)

UNITID,INSTNM,219976
UNITID,,
219976,Lipscomb University,1.000000
137847,The University of Tampa,0.989842
173045,Augsburg College,0.984756
121309,Point Loma Nazarene University,0.984000
151263,University of Indianapolis,0.983916
147679,North Park University,0.982748
159656,Loyola University New Orleans,0.982465
207458,Oklahoma City University,0.979085
181002,Creighton University,0.978569


### LEAST similar schools to Harvard:

In [34]:
cosim[['INSTNM', 439279]].sort_values(439279, ascending=False).head(30)

UNITID,INSTNM,439279
UNITID,,
439279,University of Phoenix-St Louis Campus,1.000000
450456,University of Phoenix-Birmingham Campus,0.869044
450474,University of Phoenix-Augusta Campus,0.867255
474960,University of Phoenix-Lafayette Campus,0.866714
450483,University of Phoenix-Washington DC Campus,0.866571
448567,University of Phoenix-Columbia Campus,0.866332
442161,University of Phoenix-Chicago Campus,0.865762
474951,University of Phoenix-Baton Rouge Campus,0.865282
440448,University of Phoenix-Houston Campus,0.864291


## I'm amazed how much better a job this is doing just splitting out the categorical variables!!!

# Finally, save to csv.

In [35]:
cosim.to_csv('data/similarity_index.csv')